In [ ]:
from langchain.chat_models import ChatOpenAI
chat = ChatOpenAI(temperature=0.1)

from langchain.prompts import PromptTemplate

template = PromptTemplate.from_template("{nara}의 수도는 어디인가요? 그리고 당신의 이름은 무엇인가요?")

prompt = template.format(nara="대한민국")

chat.predict(prompt)

In [ ]:
from langchain.prompts import ChatPromptTemplate

template = ChatPromptTemplate.from_messages([
    ("system", "당신은 친절한 AI 비서입니다."),
    ("ai", "안녕하세요! 저는 지영봇입니다.") ,
    ("human", "{nara}의 수도는 어디인가요? 그리고 당신의 이름은 무엇인가요?")
])

prompt = template.format_messages(nara="대한민국")

chat.predict_messages(prompt)

In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(
    temperature=0.1, streaming=True, callbacks=[StreamingStdOutCallbackHandler()]
)
chef_prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 요리사입니다. 요리에 대한 레시피를 간단하게 제안합니다."),
    ("human", "나는 {요리}를 만들기를 원합니다.")
])

veg_chef_prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 채식주의자 요리사입니다. 레시피를 채식주의에 맞게 다시 레시피를 간단하게 알려주세요."),
    ("human", "{레시피}")
])

chef_chain = chef_prompt | chat
veg_chanin = veg_chef_prompt | chat

final_chain = {"레시피":chef_chain} | veg_chanin

final_chain.invoke({"요리":"김치찌개"})